In [1]:
import numpy as np
import pandas as pd
import os
import copy
from collections import defaultdict

In [2]:
from config import *
from utils import save_result, time_test, read_data, setup

In [11]:
class Graph:
    """
    用稀疏矩阵来存储数据：src、指向src的边、出度
    此结构方便计算
    """
    def __init__(self, nnodes, edges, number_of_nodes) -> None:
        self.nnodes = nnodes  # 最大节点数
        self.number = number_of_nodes  # 图中节点的数量
        self.in_edges = []#这是 nnodes+1 维的：0到nnodes+1
        self.out_degrees = [0] * (nnodes + 1)
        for i in range(nnodes + 1):#nnodes+1 维的：0到nnodes+1
            self.in_edges.append([])

        for edge in edges:
            # edge: src, des
            # 做一个reverse操作
            self.out_degrees[edge[0]] += 1
            self.in_edges[edge[1]].append(edge[0])

In [13]:
def load_graph() -> Graph:
    # 读入数据
    # Read data from dataset
    links = []
    # 【注意改路径！】
    with open("../Data/data.txt", "r", encoding="utf-8") as file:
        for line in file:
            src, dst = map(int, line.split())
            links.append((src, dst))

        nodes=[]
        # Get unique list of nodes
        #TODO OPTIMISE
        for row in links:
            if row not in nodes:
                nodes.append(row)
        # Get unique list of nodes
        max_both = -1   #最大结点值，本实验为8297
        for i in links:
            max_both = max(max_both, i[0], i[1])
        node_list = []    
        for link in nodes:
            for node in link:
                if node not in node_list:
                    node_list.append(node)
        number_of_nodes = len(node_list)
        return Graph(max_both, nodes, number_of_nodes)

In [5]:
def pagerank2(graph: Graph):
    N = graph.nnodes
    r_old = np.full(N + 1, 1 / N)  # 初始化向量
    # print(r_old)
    r_old[0] = 0
    iter = 0
    i=0
    while True:
        r_new = np.zeros(N + 1)
        for dst in range(1, N + 1):  # dst只是个变量哈：表示从结点1：N+1
            for src in graph.in_edges[dst]:  # 指向dst的结点
                r_new[dst] += r_old[src] / graph.out_degrees[src]

        r_new *= TELEPORT
        s = float(np.sum(r_new))
        r_new += (1 - s) / N  # Now re-insert the leaked PageRank：修正
        #TODO:范式、收敛值e可改 看能否提高准确度
        e = np.linalg.norm(r_new - r_old, ord=NORM)  # 第一范式距离表示收敛
        iter += 1
        if e < EPSILON or iter >= MAX_ITER:
            print(f"absolute error: {e}, iter: {iter}")
            break
        r_old = np.copy(r_new)

    result = {}
    for i in range(1, N + 1):
        result[i] = r_new[i]

    return result

In [16]:
def pagerank2_revise(graph: Graph):
    # N = graph.nnodes+1#N表示向量维度
    N  = graph.number
    print("图中结点总数为 %d"%N)
    print("图中最大结点编号为 %d"%graph.nnodes)
    #初始化矩阵
    r_old = np.full(graph.nnodes+1, np.float64(1 / N))  # 初始化向量：graph.nnodes+1维

    i = 0
    while i < (graph.nnodes+1):
        if graph.out_degrees[i] == 0 and len(graph.in_edges[i]) == 0:  # 出度为0且入度为0
            r_old[i]=0; #保持为0
            # print("i等于%d时为孤立节点"%i)
        # print(r_old[i])
        i += 1

    s1 = float(np.sum(r_old))
    # print("初始s的值%f" % s1)

    iter = 0
    while True:
        # print("\n第%d轮迭代" % iter)
        r_new = np.zeros(graph.nnodes + 1)#和r_old保持一样的维度：graph.nnodes
        for dst in range(graph.nnodes+ 1):  # dst（是key值） 从结点0：graph.nnodes
            #TODO:有待检验这个步骤正确性以及是否多余!
            if len(graph.in_edges[dst]) == 0:  #入度为0
                r_new[dst]=0
            else:
                for src in graph.in_edges[dst]:  # 指向dst的结点：如果没有入边？
                    # print("r%d的出度为%d"%(src,graph.out_degrees[src]))
                    r_new[dst] += (r_old[src] / graph.out_degrees[src])*TELEPORT
                    # print("r_new%d更新为%.17f" %( dst,  r_new[dst]))



        #这里加和也不能往出入度为0的结点的序号上加——让那些编号对应的结点继续向量值为0
        s=float(np.sum(r_new))
        # print("修正前s的值%.17f" % s)

        i = 0
        while i < (graph.nnodes + 1):
            if graph.out_degrees[i] == 0 and len(graph.in_edges[i]) == 0:  # 出度为0且入度为0
                r_new[i] = 0;  # 对应孤立节点初始向量为0，且让其始终为0
                i += 1
                continue
            r_new[i] += (1 -s) / N  # Now re-insert the leaked PageRank：修正
            i += 1

        s = np.sum(r_new)
        # print("修正后s的值%f" % s)
        # print("修正后的向量为:")
        # print(r_new)


        e = np.linalg.norm(r_new - r_old, ord=NORM)  # 第一范数距离表示收敛
        iter += 1
        if e < EPSILON or iter >= MAX_ITER:
            print(f"absolute error: {e}, iter: {iter}")
            break
        # r_old = np.copy(r_new)
        r_old = copy.deepcopy(r_new)

    result = {}
    for i in range(1,graph.nnodes+1):
        result[i] = r_new[i]

    return result

In [ ]:
def pagerank(graph, d=0.85, max_iter=100, tol=1e-6):
    # 初始化PageRank向量
    pagerank = np.ones(graph.number) / graph.number

    # 计算每个节点的出度逆值
    out_degree_inv = np.zeros(graph.nnodes + 1)
    for i in range(1, graph.nnodes + 1):
        if graph.out_degrees[i] > 0:
            out_degree_inv[i] = 1.0 / graph.out_degrees[i]

    # 迭代计算PageRank
    for iter in range(max_iter):
        old_pagerank = pagerank.copy()

        for i in range(graph.number):
            rank = 0.0
            for j in graph.in_edges[i]:
                rank += old_pagerank[j] * out_degree_inv[j]
            pagerank[i] = (1 - d) / graph.number + d * rank

        # 判断收敛
        if np.abs(pagerank - old_pagerank).sum() < tol:
            break

    # 返回PageRank值
    return dict(zip(range(graph.nnodes), pagerank))

In [17]:
if __name__ == '__main__':
    print("This is Basic Page Rank")

    if setup() == 0:
        graph = time_test("read_graph", load_graph)  # 不断地返回边
        # graph = load_graph()# 不断地返回边

        # print(graph.out_degrees[2])
        # print(graph.in_edges[2])
        # print(len(graph.in_edges[2]))
        print(graph.nnodes)
        print(graph.number)
        # TODO:尝试着写出graph文件看数据集特点
        result = time_test("pagerank", pagerank2_revise, graph)

        save_result(result, BASIC_OUT)

        result = sorted(result.items(), key=lambda x: x[1], reverse=True)
        #仅仅为了方便统计。。。
        topn=100
        if topn > 0:
            result = result[:topn]
        filename2 = "../Results/Basic_nodeID.txt"
        filename3 = "../Results/Basic_PR_score.txt"

        with open(filename2, 'w', encoding='utf-8') as f:
            for line in  result:
                f.write(f"{line[0]:<10}\n")

        with open(filename3, 'w', encoding='utf-8') as f:
            for line in  result:
                f.write(f"{line[1]}\n")

        # os.system("pause")

This is Basic Page Rank
..\Results is exist.
Middle is exist.
图中最大结点编号为 8297
图中结点总数为 6263
read_graph:  99.50s
图中结点总数为 6263
图中最大结点编号为 8297
absolute error: 8.711517815301737e-07, iter: 44
pagerank:  1.97s
file saved


In [ ]:
# 索引
def renumber_edges(edges):
    node_dict = {}
    new_edges = []
    new_id = 0

    # 遍历所有的边，建立新旧节点编号的映射关系
    for edge in edges:
        src, dst = edge

        if src not in node_dict:
            node_dict[src] = new_id
            new_id += 1

        if dst not in node_dict:
            node_dict[dst] = new_id
            new_id += 1

        new_edges.append((node_dict[src], node_dict[dst]))

    return new_edges, node_dict

# 构造一个有六千多个节点的图
edges = [(1, 2), (1, 3), (2, 1), (3, 2), (3, 4), (4, 2), (4, 5), (5, 1), (5, 4)]
for i in range(6, 8231):
    edges.append((i, i-5))

# 对边进行重新编号
renumbered_edges, node_dict = renumber_edges(edges)

# 构造图结构
graph = Graph(len(node_dict), renumbered_edges, len(node_dict))

# 计算PageRank
pagerank_dict = pagerank(graph)

# 将新节点编号映射回原节点编号
original_node_id = {v: k for k, v in node_dict.items()}
original_pagerank_dict = {original_node_id[new_id]: rank for new_id, rank in pagerank_dict.items()}

# 输出每个节点的PageRank值
for node, rank in original_pagerank_dict.items():
    print("Node %d: PageRank = %.5f" % (node, rank))
